# Recolectar información del clima
Curso de Python Científico versión 6 - https://github.com/badillosoft/python-scig#recolectar-informaci%C3%B3n-del-clima-1pt

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import Selector

In [1]:
import json

class JsonWriterPipeline(object):
    
    def open_spider(self, spider):
        self.file = open('quoteresult.jl','w')
        
    def close_spider(self, spider):
        self.file.close()
        
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
# Spider direccionado a la página de weather


class ClimaSpider(scrapy.Spider):
    name = "cdmx"
    start_urls = ["https://weather.com/es-MX/tiempo/hoy/l/MXDF0132:1:MX"]
    custom_settings = {
        'ITEM_PIPELINES' : {'__main__.JsonWriterPipeline': 1}, 
        'FEED_FORMAT' : 'json',
        'FEED_URI' : 'cdmx-%(time)s.json'
    }
    def parse(self, response):
        for clima in response.css("div.today_nowcard-section"):
            yield {
                "temp": clima.css("div.today_nowcard-temp span::text").extract_first(),   # Extrae temperatura
                "desc": clima.css("div.today_nowcard-phrase::text").extract_first(),      # Extrae descripción
                "feel": clima.css("div.today_nowcard-feels span.deg-feels::text").extract_first(),           # Extrae sensación
                "max_min": clima.css("div.today_nowcard-hilo span.deg-hilo-nowcard::text").extract_first(),  # Extrae máxima y mínima
                "indiceUV": clima.css("div.today_nowcard-hilo::text").extract_first()                        # Extrae índice UV
            }

In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(ClimaSpider)
process.start()

2018-12-20 19:22:28 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2018-12-20 19:22:28 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.19.0, Twisted 17.5.0, Python 2.7.15 | packaged by conda-forge | (default, Jul 27 2018, 10:23:44) - [GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)], pyOpenSSL 18.0.0 (OpenSSL 1.0.2p  14 Aug 2018), cryptography 2.3.1, Platform Darwin-14.5.0-x86_64-i386-64bit
2018-12-20 19:22:28 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'cdmx-%(time)s.json', 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2018-12-20 19:22:28 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2018-12-20 19:22:28 [scrapy.middleware] INFO: Enabled down